# Getting Started with Influx and Pandas
## This is a simple tutorial to get you querying Influx, converting your time series data into a DataFrame, and converting your dataframe back into line protocol using the [Influx Python Client Library](https://github.com/influxdata/influxdb-client-python). 

### ASSUMPTIONS: you have already configured telegraf to gather system stats on your machine.
### CONSIDERATIONS: the conversion to line protocol isn't part of the client. 

### STEP ONE: import dependencies 

In [10]:
from influxdb_client import InfluxDBClient
import pandas as pd
from dataframe_to_line_protocol import lp

### STEP TWO: Supply auth parameteres 

In [11]:
my_token = "$mytoken"
my_org = "my-org"
bucket = "system"

### STEP THREE: Write your flux query 

In [12]:
query='''
from(bucket: "system")
  |> range(start:-5m, stop: now())
  |> filter(fn: (r) => r._measurement == "cpu")
  |> filter(fn: (r) => r._field == "usage_user")
  |> filter(fn: (r) => r.cpu == "cpu-total")
'''

### STEP FOUR: Query Influx and return a DataFrame

In [13]:
client = InfluxDBClient(url="http://localhost:9999", token=my_token, org=my_org, debug=False)

system_stats = client.query_api().query_data_frame(org=my_org, query=query)
system_stats.head()

,result,table,_start,_stop,_time,_value,_field,_measurement,cpu,host
0,_result,0,2020-02-21 18:10:44.436471+00:00,2020-02-21 18:15:44.436471+00:00,2020-02-21 18:14:50+00:00,4.811297,usage_user,cpu,cpu-total,Anais.attlocal.net
1,_result,0,2020-02-21 18:10:44.436471+00:00,2020-02-21 18:15:44.436471+00:00,2020-02-21 18:15:00+00:00,7.497189,usage_user,cpu,cpu-total,Anais.attlocal.net
2,_result,0,2020-02-21 18:10:44.436471+00:00,2020-02-21 18:15:44.436471+00:00,2020-02-21 18:15:10+00:00,10.095071,usage_user,cpu,cpu-total,Anais.attlocal.net
3,_result,0,2020-02-21 18:10:44.436471+00:00,2020-02-21 18:15:44.436471+00:00,2020-02-21 18:15:20+00:00,6.034483,usage_user,cpu,cpu-total,Anais.attlocal.net
4,_result,0,2020-02-21 18:10:44.436471+00:00,2020-02-21 18:15:44.436471+00:00,2020-02-21 18:15:30+00:00,6.100000,usage_user,cpu,cpu-total,Anais.attlocal.net


### STEP FIVE: convert to line protocol 


In [53]:
lines = lp(system_stats,"_measurement","cpu","_field","_value","_time")
lines[0:5]

['cpu,cpu=cpu-total usage_user=4.4630578822352795 1576013660000000000',
 'cpu,cpu=cpu-total usage_user=3.1875 1576013670000000000',
 'cpu,cpu=cpu-total usage_user=3.7735849056603774 1576013680000000000',
 'cpu,cpu=cpu-total usage_user=3.3020637898686678 1576013690000000000',
 'cpu,cpu=cpu-total usage_user=4.77440319960005 1576013700000000000']

### STEP SIX: write points to Influx

In [ ]:
from influxdb_client import InfluxDBClient, Point, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS

_write_client = client.write_api(write_options=WriteOptions(batch_size=1000, 
                                                            flush_interval=10_000,
                                                            jitter_interval=2_000,
                                                            retry_interval=5_000))
# Preparing Dataframe: 
system_stats.drop(columns=['result', 'table','start','stop'])
# DataFrame must have the timestamp column as an index for the client. 
system_stats.set_index("_time")
_write_client.write(bucket.name, record=system_stats, data_frame_measurement_name='cpu',
                    data_frame_tag_columns=['cpu'])

In [47]:
#To close the client 
_write_client.__del__()
client.__del__()